# 0. 입지선정후보군데이터 만들기
* 공공체육시설 데이터 + 공원 데이터
* 행정동 / 분류 / 시설명 / x좌표(127.xx) / y좌표(37.xx)

# 1. 공공체육시설데이터에서 행정동 뽑기

In [100]:
import pandas as pd
df1 = pd.read_csv('공공체육시설데이터.csv', encoding='cp949')

### 칼럼 정리

In [101]:
df1.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'FACI_NM', 'FACI_GB_CD', 'FACI_GB_NM',
       'FCOB_CD', 'FCOB_NM', 'FTYPE_CD', 'FTYPE_NM', 'FACI_STAT',
       'FACI_ROAD_POST', 'FACI_ROAD_ADDR1', 'FACI_ROAD_ADDR2', 'FACI_POST',
       'FACI_ADDR1', 'FACI_ADDR2', 'FACI_POINT_X', 'FACI_POINT_Y', '서울유무'],
      dtype='object')

In [102]:
df1 = df1[['FACI_ADDR1', 'FCOB_NM', 'FACI_NM', 'FACI_POINT_X', 'FACI_POINT_Y']]
df1 = df1.rename(columns={'FACI_ADDR1': '행정동', 
                         'FCOB_NM': '분류',
                         'FACI_NM': '시설명',
                         'FACI_POINT_X': 'x좌표', 
                         'FACI_POINT_Y': 'y좌표'})
df1.head()

,행정동,분류,시설명,x좌표,y좌표
0,서울특별시 성북구 석관동,간이운동장,돌 뫼,127.052763,37.605125
1,서울특별시 성북구 장위1동,간이운동장,동 방,127.043551,37.616625
2,서울특별시 성북구 월곡1동,간이운동장,동 산,127.032290,37.605577
3,서울특별시 성북구 월곡1동,간이운동장,밤 골,127.039928,37.607571
4,서울특별시 성북구 삼선동,간이운동장,삼 선,127.008989,37.582602


### 행정동 추출

In [103]:
def hangjungdong(x):
    return x.split()[2]

In [104]:
df1['행정동'] = df1['행정동'].apply(hangjungdong)

### 결측치 제거

In [105]:
df1 = df1.dropna(how='any').reset_index(drop=True)

# 2. 공원데이터에서 행정동 뽑기

In [106]:
import pandas as pd
df2 = pd.read_csv('공원데이터.csv', encoding='cp949')

### 서울만 가져오기

In [107]:
df2 = df2[df2['서울유무'] == '서울']

### 칼럼 추출

In [108]:
df2.columns

Index(['관리번호', '공원명', '공원구분', '소재지도로명주소', '소재지지번주소', '위도', '경도', '공원면적',
       '공원보유시설(운동시설)', '공원보유시설(유희시설)', '공원보유시설(편익시설)', '공원보유시설(교양시설)',
       '공원보유시설(기타시설)', '지정고시일', '관리기관명', '전화번호', '데이터기준일자', '제공기관코드', '제공기관명',
       '서울유무', '행정동'],
      dtype='object')

In [109]:
df2 = df2[['소재지지번주소', '공원구분', '공원명', '경도', '위도']]
df2 = df2.rename(columns={'소재지지번주소': '행정동',
                         '공원구분': '분류',
                         '공원명': '시설명',
                         '경도': 'x좌표',
                         '위도': 'y좌표'})
df2.head()

,행정동,분류,시설명,x좌표,y좌표
170,서울특별시 강남구 논현동 263-23,어린이공원,학동공원,127.038764,37.511646
171,서울특별시 강남구 신사동 558,어린이공원,신사은행나무공원,127.021627,37.521114
172,서울특별시 강남구 신사동 567-37,어린이공원,신사목련공원,127.017064,37.522898
173,서울특별시 강남구 신사동 625-5,어린이공원,신사무궁화공원,NaN,37.524131
174,서울특별시 강남구 신사동 597,어린이공원,신사까치공원,127.020466,NaN


### 행정동 추출

In [110]:
df2['행정동'] = df2['행정동'].apply(hangjungdong)

### 결측치 제거

In [111]:
df2.isnull().sum()

행정동     0
분류      0
시설명     0
x좌표    73
y좌표    70
dtype: int64

In [112]:
df2 = df2.dropna(how='any').reset_index(drop=True)

In [113]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1752 entries, 0 to 1751
Data columns (total 5 columns):
행정동    1752 non-null object
분류     1752 non-null object
시설명    1752 non-null object
x좌표    1752 non-null float64
y좌표    1752 non-null float64
dtypes: float64(2), object(3)
memory usage: 68.5+ KB


# 3. 행정동 전처리

In [114]:
df = pd.concat([df1, df2], axis=0)
#df.to_csv('입지선정후보군데이터.csv', encoding='cp949')

### '동'을 가진 행만 추출

In [98]:
df = df[df['행정동'].str.contains('동')]

### 숫자 있으면 숫자 제거

In [99]:
import re
def removenumber(x):
    p = re.compile("[^0-9]")
    res = "".join(p.findall(x))
    return res

In [74]:
df['행정동'] = df['행정동'].apply(removenumber)

### '가'로 끝나는거 처리

In [75]:
# def removega(x):
#     res = x.replace('가', "")
#     return res

In [76]:
# df['행정동'] = df['행정동'].apply(removega)

### 그 외 특문 처리

In [77]:
def removechar(x):
    chars = [".", "-"]
    for char in chars:
        res = x.replace(char, "")
    return res

In [78]:
df['행정동'] = df['행정동'].apply(removechar)

In [81]:
# 종로...동 삭제
df = df[df['행정동'] != '종로...가동']

In [82]:
df['행정동'].unique()

array(['석관동', '장위동', '월곡동', '삼선동', '종암동', '정릉동', '아현동', '서교동', '쌍문동',
       '창동', '신원동', '혜화동', '천연동', '홍제동', '신림동', '보문동', '반포동', '암사동',
       '안암동', '돈암동', '길음동', '영등포동', '방이동', '사직동', '시흥동', '신수동', '월계동',
       '대흥동', '도봉동', '상계.동', '창전동', '공덕동', '도화동', '방학동', '연남동', '개봉동',
       '성산동', '신도림동', '고척동', '북아현동', '방배동', '광장동', '오류동', '양재동', '공항동',
       '서초동', '반포본동', '내곡동', '가양동', '일원동', '압구정동', '사당동', '상도동', '흑석동',
       '대방동', '보라매동', '노량진동', '도곡동', '신월동', '방화동', '방배본동', '잠원동', '역삼동',
       '염창동', '우장산동', '화곡동', '등촌동', '화곡본동', '면목동', '중화동', '이화동', '논현동',
       '상수동', '상암동', '개포동', '독산동', '장충동', '성수동가', '목동', '중계본동', '중계동',
       '고덕동', '세곡동', '용강동', '잠실동', '옥인동', '현석동', '효창동', '후암동', '망원동',
       '자양동', '구로동', '진관동', '상일동', '봉천동', '대치동', '신사동', '마장동', '신공덕동',
       '삼성동', '신정동', '응암동', '상계동', '불광동', '사근동', '공릉동', '송파동', '오금동',
       '한남동', '응봉동', '홍은동', '장충동가', '중계.동', '청담동', '하계동', '수서동', '염리동',
       '청파동', '본동', '번동', '구의동', '갈현동', '녹번동', '당인동', '대조동', '동교동',
       '

In [42]:
df.head()

,행정동,분류,시설명,x좌표,y좌표
0,석관동,간이운동장,돌 뫼,127.052763,37.605125
1,장위동,간이운동장,동 방,127.043551,37.616625
2,월곡동,간이운동장,동 산,127.032290,37.605577
3,월곡동,간이운동장,밤 골,127.039928,37.607571
4,삼선동,간이운동장,삼 선,127.008989,37.582602


### 데이터 내보내기

In [43]:
df.to_csv('입지선정후보군데이터.csv', encoding='cp949', index=None)

# 4. 추가 엑셀 작업

### 입지선정후보군_행정동추가

In [116]:
a = pd.read_csv("입지선정후보군데이터.csv", encoding='cp949')
a.head()

,Unnamed: 0,분류,시설명,x좌표,y좌표,법정동,행정동
0,0,간이운동장,돌 뫼,127.052763,37.605125,석관동,석관동
1,1,간이운동장,동 방,127.043551,37.616625,장위동,장위제1동
2,2,간이운동장,동 산,127.032290,37.605577,하월곡동,길음제2동
3,3,간이운동장,밤 골,127.039928,37.607571,하월곡동,길음제2동
4,4,간이운동장,삼 선,127.008989,37.582602,삼선동1가,삼선동


In [124]:
a = a[['분류', '시설명', 'x좌표', 'y좌표', '법정동', '행정동']]
b = a[['분류', '시설명', '행정동']]

In [128]:
a.to_csv('입지선정후보군_좌표추가.csv', encoding='cp949', index=None)
b.to_csv('입지선정후보군_행정동추가.csv', encoding='cp949', index=None)

In [123]:
a.isnull().sum()
a = a.dropna(how='any').reset_index(drop=True)